In [8]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers.language import LanguageParser
from langchain_text_splitters import RecursiveCharacterTextSplitter, Language
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain.chains.question_answering import load_qa_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from pathlib import Path


In [9]:
import os
from git import Repo

In [10]:
repo_path = Path("/Users/yan/Documents/GitHub/Python/LogicaDeProgramação/MachineLearning/RAG/repo_path")

In [11]:
repo = Repo.clone_from("https://github.com/langchain-ai/langchain", to_path=str(repo_path)) if not (repo_path / ".git").exists() else Repo(str(repo_path))


In [12]:
# Carrega arquivos .py do repositório usando um parser de linguagem
loader = GenericLoader.from_filesystem(
    # Diretório raiz que contém o código-fonte a ser indexado
    str(repo_path / "libs/core/langchain_core"),
    # Padrão de busca: percorre recursivamente todos os caminhos
    glob = "**/*",
    # Extensões incluídas: apenas arquivos Python
    suffixes = [".py"],
    # Arquivos excluídos: evita problemas de encoding
    exclude = ["**/non-utf-8-encoding.py"],
    # Parser para extrair funções/classes e separar conteúdo
    # 'parser_threshold=500' evita segmentação quando o arquivo é pequeno
    parser = LanguageParser(language="python", parser_threshold=500)
)
# Realiza a leitura e retorna uma lista de Document
documents = loader.load()
len(documents)


569

In [13]:
# Divide os documentos Python em chunks para melhor recuperação
python_splitter = RecursiveCharacterTextSplitter.from_language(
    # Linguagem alvo: enum Language.PYTHON (pode ser 'python')
    language=Language.PYTHON,
    # Tamanho máximo do chunk em caracteres
    chunk_size=2000,
    # Sobreposição entre chunks para preservar contexto
    chunk_overlap=200
)

# Aplica a divisão e contabiliza o total de chunks
texts = python_splitter.split_documents(documents)
len(texts)


1591

In [14]:
# Ollama não requer chave de API. Este passo não é necessário.
# Caso esteja usando OpenAI em outro contexto, prefira variáveis de ambiente/.env
# Exemplo (não necessário aqui): os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [15]:
# Cria a base vetorial (índice) em Chroma a partir dos textos
# Usa embeddings locais via Ollama; 'nomic-embed-text' é modelo de embeddings
# Dica: rode 'ollama pull nomic-embed-text' antes, se necessário
import os
os.environ["OLLAMA_HOST"] = "http://localhost:11434"
persist_dir = "./chroma_db"
collection_name = "langchain_core_py"
emb = OllamaEmbeddings(model="nomic-embed-text", base_url="http://localhost:11434")
# Valida conexão com Ollama e o modelo de embeddings
_ = emb.embed_query("ping")
db = Chroma.from_documents(
    texts,
    emb,
    persist_directory=persist_dir,
    collection_name=collection_name
)

# Transforma a base em um 'retriever' para buscar contexto relevante
retriever = db.as_retriever(
    # 'mmr' (Maximal Marginal Relevance): equilibra relevância e diversidade
    # Evita retornar documentos muito redundantes, aumentando a cobertura temática
    search_type="mmr",
    # 'k' define o número de documentos finais retornados pelo retriever
    # Valores comuns: 4–8; maiores valores aumentam recall mas podem trazer redundância
    search_kwargs={"k": 8}
)


ValueError: Error raised by inference API HTTP code: 500, {"error":"do embedding request: Post \"http://127.0.0.1:53830/embedding\": EOF"}

In [16]:
# Configura o modelo local via Ollama para gerar respostas
# Usando o modelo 'gemma3:4b' (precisa estar baixado no Ollama)
# Dica: rode 'ollama pull gemma3:4b' antes, se necessário
llm = ChatOllama(
    model="gemma3:4b",
    base_url="http://localhost:11434",
    temperature=0
)


/var/folders/5l/x5tjl9qn4kq7zrm6vh3twr300000gn/T/ipykernel_66397/2910614059.py:4: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(


In [17]:
# Define o prompt do chat com mensagens de sistema e usuário
prompt = ChatPromptTemplate.from_messages([
    # Mensagem de sistema: orienta o comportamento do assistente e injeta o contexto
    ("system", "You are a helpful assistant that answers questions about the document. {context}"),
    # Mensagem de usuário: consulta que será respondida pelo LLM
    ("user", "{input}"),
])

# Cadeia que 'stuffa' os documentos no prompt do LLM
document_chain = create_stuff_documents_chain(llm, prompt)
# Cadeia de recuperação: busca via retriever e passa contexto para a cadeia de documentos
if 'retriever' not in globals():
    raise RuntimeError("Retriever não definido. Execute a célula de criação do índice (Chroma) e garanta que o Ollama esteja ativo e o modelo 'nomic-embed-text' baixado.")
retrieval_chain = create_retrieval_chain(retriever, document_chain)


NameError: name 'retriever' is not defined

In [ ]:
# Executa a cadeia de recuperação com a pergunta do usuário
response = retrieval_chain.invoke({"input": "Qual é o objetivo do projeto?"})


In [ ]:
# Exibe a resposta final gerada pelo LLM, já com contexto
print(response["answer"])
